In [ ]:
# https://github.com/AlaGrine/LinkedIn_Job_Scraper_and_Matcher/tree/main/FLASK_app
# pip install selenium 
# pip install beautifulsoup4

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import requests

import time, datetime
import pandas as pd
import numpy as np
import math, re, sys
import warnings
warnings.filterwarnings("ignore")

# 1. Instanciate the chrome service
chromedriver_path = 'C:\OAMK\Python Programs\chromedriver-win64130\chromedriver-win64\chromedriver.exe'
service = Service(executable_path=chromedriver_path)

# 2. Instanciate the webdriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-search-engine-choice-screen")
driver = webdriver.Chrome(options=options, service=service)

# 3. Open the LinkedIn login page
driver.get('https://www.linkedin.com/login')
time.sleep(5) # waiting for the page to load

# 4. Enter email address & password
email_input = driver.find_element(By.ID, 'username')
password_input = driver.find_element(By.ID, 'password')
email_input.send_keys("xx@xx.com")
password_input.send_keys("xx")

# 5. Click the login button
password_input.send_keys(Keys.ENTER)

time.sleep(10)

List_Job_IDs = []

# Create a function 'Scroll to the bottom'. 

# time.sleep() function is used to provide extra time for the webpage to load. 
# I used 120 seconds. If the 25 jobs have not loaded during this period, we can make adjust it and test again.

def scroll_to_bottom(driver,sleep_time=120):
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            break
        last_height = new_height
    
    time.sleep(sleep_time)  



# Navigate to the first page (start=0) and scroll to the bottom of the page

keywords = 'data'
location = 'Finland'
start = 0

url = f'https://www.linkedin.com/jobs/search/?keywords={keywords}&location={location}&start={start}'
url = requests.utils.requote_uri(url)
driver.get(url)
scroll_to_bottom(driver,sleep_time=120)


# Get number of jobs found and number of pages:

# Parse the HTML content of the page using BeautifulSoup.
soup = BeautifulSoup(driver.page_source, 'html.parser')

try:
    div_number_of_jobs = soup.find("div",{"class":"jobs-search-results-list__subtitle"})
    number_of_jobs = int(div_number_of_jobs.find('span').get_text().strip().split()[0])
except:
    number_of_jobs = 0
    
number_of_pages=math.ceil(number_of_jobs/25)
print("number_of_jobs:",number_of_jobs)
print("number_of_pages:",number_of_pages)



# Get Job Ids present on the first page.

def find_Job_Ids(soup):

    Job_Ids_on_the_page = []
    
    job_postings = soup.find_all('li', {'class': 'jobs-search-results__list-item'})
    for job_posting in job_postings:
        Job_ID = job_posting.get('data-occludable-job-id')
        Job_Ids_on_the_page.append(Job_ID)
        # job_title = job_posting.find('a', class_='job-card-list__title').get_text().strip()
        # location = job_posting.find('li', class_='job-card-container__metadata-item').get_text().strip()
    
    return Job_Ids_on_the_page    

Jobs_on_this_page = find_Job_Ids(soup)
List_Job_IDs.extend(Jobs_on_this_page)


if number_of_pages>1:
    
    for page_num in range(1,number_of_pages):
        print(f"Scraping page: {page_num}",end="...")
        
        # Navigate to page
        url = f'https://www.linkedin.com/jobs/search/?keywords={keywords}&location={location}&start={25 * page_num}'
        url = requests.utils.requote_uri(url)
        driver.get(url)
        scroll_to_bottom(driver)

        # Parse the HTML content of the page using BeautifulSoup.
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Get Job Ids present on the page.
        Jobs_on_this_page = find_Job_Ids(soup)
        List_Job_IDs.extend(Jobs_on_this_page)  
        print(f'Jobs found:{len(Jobs_on_this_page)}')

pd.DataFrame({"Job_Id":List_Job_IDs}).to_csv('Job_Ids.csv',index=False)

## Close the browser and shut down the ChromiumDriver executable that
# is started when starting the ChromiumDriver. 
driver.quit()





In [2]:
def remove_tags(html):
    '''remove html tags from BeautifulSoup.text'''
 
    soup = BeautifulSoup(html, "html.parser")
 
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
 
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)
  
job_url='https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'
job={}
list_jobs=[]

for j in range(0,len(List_Job_IDs)):
    print(f"{j+1} ... read jobId:{List_Job_IDs[j]}")

    resp = requests.get(job_url.format(List_Job_IDs[j]))
    soup=BeautifulSoup(resp.text,'html.parser')
    # print(soup.prettify()) 

    job["Job_ID"] = List_Job_IDs[j] 
    
    try: 
        job["Job_txt"] = remove_tags(resp.content)
    except:
        job["Job_txt"] = None
    
    try:
        job["company"]=soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
    except:
        job["company"]=None

    try:
        job["job-title"]=soup.find("div",{"class":"top-card-layout__entity-info"}).find("a").text.strip()
    except:
        job["job-title"]=None

    try:
        job["level"]=soup.find("ul",{"class":"description__job-criteria-list"}).find("li").text.replace("Seniority level","").strip()
    except:
        job["level"]=None

    try:
        job["location"]=soup.find("span",{"class":"topcard__flavor topcard__flavor--bullet"}).text.strip()
    except:
        job["location"]=None

    try:
        job["posted-time-ago"]=soup.find("span",{"class":"posted-time-ago__text topcard__flavor--metadata"}).text.strip()
    except:
        job["posted-time-ago"]=None

    try:
        nb_candidats = soup.find("span",{"class":"num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
        nb_candidats = int(nb_candidats.split()[0])
        job["nb_candidats"]= nb_candidats
    except:
        job["nb_candidats"]=None

    list_jobs.append(job)
    job={}

# create a pandas Datadrame
jobs_df = pd.DataFrame(list_jobs)

In [3]:
from openpyxl import Workbook

#jobs_df.to_csv("jobsFrame.csv")
#jobs_df.to_excel("jobsFrame2.xlsx")
jobs_df.to_excel("jobsFrame4.xlsx")

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import requests

import time, datetime
import pandas as pd
import numpy as np
import math, re, sys
import warnings
import selenium
import bs4

print(pd.__version__)
print(selenium.__version__)
print(bs4.__version__)
print(requests.__version__)
print(np.__version__)



In [ ]:
soup